In [4]:
# Standard libraries
import os
import sys
import json
from dotenv import load_dotenv

# Third-party libraries
import pandas as pd

# Add project root to sys.path
project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

# Local modules
from src.classifiers import is_uae_real_estate_query, llm_classifier
from src.tools import extract_data_intent, safe_dataframe_tool, create_plotly_code,get_openai_llm
from src.geo_tools import generate_google_maps_html

load_dotenv()

True

In [12]:
query = extract_data_intent.invoke("show histogram for the properties prices in this uae database and set the number of bins to 15")

In [16]:
from openai.types.chat import ChatCompletionSystemMessageParam, ChatCompletionUserMessageParam
from prompts.tool_prompts import get_plotly_code_prompt


In [17]:
result = safe_dataframe_tool.invoke(query)

Dataset loaded successfully.


In [23]:
df = pd.DataFrame(json.loads(result)["result"])

In [28]:
user_input = query
code_prompt = get_plotly_code_prompt(user_input, df)
# Get AI response
official_ai = get_openai_llm()
response = official_ai.chat.completions.create(
model="gpt-4.1-mini",
        messages=
        [
            ChatCompletionSystemMessageParam(role="system", content="You create Plotly code based on user_input and data"),
            ChatCompletionUserMessageParam(role="user", content=code_prompt)
        ],
        max_tokens=400
    )

# Extract code
raw_response = response.choices[0].message.content.strip()

In [31]:
from typing import Optional

def extract_python_code(text: str) -> Optional[str]:
    """
    Extract Python code from Markdown code blocks or HTML code tags.
    Returns None if no code found.

    Handles these cases:
    1. ```python\ncode\n```
    2. ```\ncode\n```
    3. <code>\ncode\n</code>
    """
    # Case 1: Python markdown
    if '```python' in text:
        return text.split('```python')[1].split('```')[0].strip()
    # Case 2: Regular markdown
    elif '```' in text:
        return text.split('```')[1].split('```')[0].strip()
    # Case 3: HTML code tags
    elif '<code>' in text.lower():
        return text.split('<code>')[1].split('</code>')[0].strip()
    return None

In [29]:
raw_response

'<code>\nimport plotly.graph_objects as go\n\nprice_ranges = [\n    "125000-13383333", "13383333-26641666", "26641666-39900000", "39900000-53158333",\n    "53158333-66416666", "66416666-79675000", "79675000-92933333", "92933333-106191666",\n    "106191666-119450000", "119450000-132708333", "132708333-145966666", "145966666-159225000",\n    "159225000-172483333", "172483333-185741666", "185741666-199000000"\n]\n\ncounts = [516, 30, 8, 3, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]\n\nfig = go.Figure(data=[\n    go.Bar(x=price_ranges, y=counts)\n])\n\nfig.update_layout(\n    title="Distribution of Property Prices in UAE Database",\n    xaxis_title="Price Range (AED)",\n    yaxis_title="Count of Properties",\n    xaxis_tickangle=-45\n)\n\nfig.show()\n</code>'

In [33]:
code = extract_python_code(response.choices[0].message.content.strip())

In [34]:
code

'import plotly.graph_objects as go\n\nprice_ranges = [\n    "125000-13383333", "13383333-26641666", "26641666-39900000", "39900000-53158333",\n    "53158333-66416666", "66416666-79675000", "79675000-92933333", "92933333-106191666",\n    "106191666-119450000", "119450000-132708333", "132708333-145966666", "145966666-159225000",\n    "159225000-172483333", "172483333-185741666", "185741666-199000000"\n]\n\ncounts = [516, 30, 8, 3, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]\n\nfig = go.Figure(data=[\n    go.Bar(x=price_ranges, y=counts)\n])\n\nfig.update_layout(\n    title="Distribution of Property Prices in UAE Database",\n    xaxis_title="Price Range (AED)",\n    yaxis_title="Count of Properties",\n    xaxis_tickangle=-45\n)\n\nfig.show()'

In [35]:
code = code.replace("fig.show()", "")

In [36]:
code

'import plotly.graph_objects as go\n\nprice_ranges = [\n    "125000-13383333", "13383333-26641666", "26641666-39900000", "39900000-53158333",\n    "53158333-66416666", "66416666-79675000", "79675000-92933333", "92933333-106191666",\n    "106191666-119450000", "119450000-132708333", "132708333-145966666", "145966666-159225000",\n    "159225000-172483333", "172483333-185741666", "185741666-199000000"\n]\n\ncounts = [516, 30, 8, 3, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]\n\nfig = go.Figure(data=[\n    go.Bar(x=price_ranges, y=counts)\n])\n\nfig.update_layout(\n    title="Distribution of Property Prices in UAE Database",\n    xaxis_title="Price Range (AED)",\n    yaxis_title="Count of Properties",\n    xaxis_tickangle=-45\n)\n\n'

In [12]:
import plotly.express as px
import pandas as pd

# Sample data based on the given input (Price Range as strings for x-axis labels)
data = {
    "Price Range": [
        "Range 0", "Range 1", "Range 2", "Range 3", "Range 4", "Range 5",
        "Range 6", "Range 7", "Range 8", "Range 9", "Range 10", "Range 11",
        "Range 12", "Range 13", "Range 14"
    ],
    "Count": [515, 30, 8, 3, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]
}

df = pd.DataFrame(data)

# Create the histogram bar chart
fig = px.bar(df, x="Price Range", y="Count",
             labels={"Count": "Number of Properties", "Price Range": "Price Range"},
             title="Histogram of Property Prices in UAE")

fig.update_layout(xaxis_title="Price Range Bin",
                  yaxis_title="Count",
                  bargap=0.2)

fig.show()

In [20]:
data = pd.read_parquet("/home/zcemg08/projects/chatbot/datasets/my-org/clean2/data.parquet")

In [22]:
data[(data["City"] == "Dubai") &  (data["addedOn_year_month"] == "2024-03") & (data["bedrooms"] == 1)]

,title,bathrooms,bedrooms,type,price,verified,priceDuration,sizeMin,furnishing,description,latitude,longitude,City,addedOn_year_month,addedOn_month_name,addedOn_month_num,addedOn_day_num,Area
316,Fully Furnished | Large Terrace | High ROI,1,1,Residential for Sale,830000,True,sell,32,YES,Lacon International is delighted to present th...,25.004149,55.286234,Dubai,2024-03,March,3,16,Town Square
394,FIVE Specialist | 10% ROI | Residential Resell,1,1,Residential for Sale,5200000,True,sell,32,YES,Residential Tower One Bedroom RESELL. Project ...,25.080890,55.136774,Dubai,2024-03,March,3,27,Jumeirah Beach Residence
520,Desirably Located | Dubai Marina | Modern buil...,2,1,Residential for Sale,1899000,True,sell,32,NO,Le-Pax Real Estate delighted to present to you...,25.068635,55.131475,Dubai,2024-03,March,3,21,Dubai Marina


In [35]:
data["Area"].unique()

array(['Business Bay', 'Jumeirah Beach Residence', 'Al Furjan', 'Meadows',
       'Dubai Marina', 'Villa', 'Springs', 'Dubai Silicon Oasis',
       'Dubai Land', 'Jumeirah Village', 'Hayyan', 'Jebel Ali',
       'Al Reem Island', 'Damac Hills', 'Arabian Ranches', 'Al Barari',
       'Khor Fakkan', 'Palm Jumeirah', 'City Walk', 'Downtown Dubai',
       'Culture Village', 'Dubai Sports City', 'Muwaileh',
       'Dubai Hills Estate', 'Jumeirah', 'Lakes',
       'Dubai Residence Complex', 'Al Jubail Island', 'Manama',
       'Al Riqqa', 'Al Ghadeer', 'Discovery Gardens', 'Al Shamkha',
       'Zabeel', 'Views', 'Difc', 'Al Nahda', 'Town Square', 'Meydan',
       'Motor City', 'Dubai Festival City', 'Majan', 'Nad Al Sheba',
       'Al Jaddaf', 'International City', 'Al Marjan Island', 'Al Warsan',
       'Wadi Al Safa', 'Barsha Heights', 'Al Wasl', 'Al Mamzar',
       'Wasl Gate', 'Dubai Industrial City', 'Al Dhait', 'Al Qurm',
       'Yas Island', 'Dubai Harbour', 'Al Tallah', 'Saadiyat Isl

In [26]:
df1 = pd.DataFrame(data_dict["result"])

In [27]:
df1

,addedOn_year_month,property_count
0,2023-09,1
1,2024-02,2
2,2024-03,2
3,2024-04,1
4,2024-05,10
5,2024-06,13
6,2024-07,46
7,2024-08,28


In [12]:
df1["latitude"].min()

25.3071588

In [8]:
query = 'Data Intent:\nThe final table should include properties that are located north to Dubai Mall and are in the 30th percentile in that group. The table should include the following columns: title, bathrooms, bedrooms, type, price, verified, priceDuration, sizeMin, furnishing, description, latitude, longitude, City, addedOn_year_month, addedOn_month_name, addedOn_month_num, addedOn_day_num, and Area. The rows should be grouped by the location (north to Dubai'

In [5]:
sample = pd.read_parquet("../data/sample_data.parquet")

In [10]:
sample.dtypes

title                  object
bathrooms               int64
bedrooms                int64
type                   object
price                   int64
verified                 bool
priceDuration          object
sizeMin                 int64
furnishing             object
description            object
latitude              float64
longitude             float64
City                   object
addedOn_year_month     object
addedOn_month_name     object
addedOn_month_num       int64
addedOn_day_num         int64
Area                   object
dtype: object

In [9]:
sample.to_dict(orient="records")

[{'title': 'Great ROI I High Floor I Creek View',
  'bathrooms': 3,
  'bedrooms': 2,
  'type': 'Residential for Sale',
  'price': 2500000,
  'verified': True,
  'priceDuration': 'sell',
  'sizeMin': 44,
  'furnishing': 'NO',
  'description': 'MNA Properties is delighted to offer this apartment located in the well-esteemed Binghatti Canal, situated in the heart of Business Bay, Dubai, UAE. The 2BR apartment with the most attractive layout based on the high floor and offers the breathtaking panoramic view of Business Bay Canal; near Burj Khalifa.\n\nTo arrange a viewing or to find out more, please contact Amir via phone or Whatsapp on +971581684981 or email at amir.m@mnaproperties.ae\n\nProperty Features:\n2 Bedrooms\n2 Bathrooms + Guest room\nSize 1,323 sq. ft.\n2 Balconies\nPrivate Jacuzzi / Small Pool\nCanal View\n\nCan be sold Unfurnished as well\n\nBinghatti Canal Residences is a newly built project in Business Bay, Dubai that features symbolic design elements to generate a sense of

In [ ]:
from src.agent import main_agent

In [ ]:
main_agent("show on map 2 bedroom flats that are cheaper than 2 million and in Dubai")

In [ ]:
USER_QUERY = "show on map 2 bedroom flats that are cheaper than 2 million and in Dubai"

command = extract_data_intent.invoke(USER_QUERY)

In [ ]:
command

In [ ]:
data_json_str = safe_dataframe_tool.invoke(command)
data_dict = json.loads(data_json_str)

In [ ]:
pd.DataFrame(data_dict["result"])['latitude']

In [ ]:
pd.DataFrame(data_dict["result"])

In [ ]:
data_json_str

In [ ]:
data_dict


In [ ]:
pd.DataFrame(data_dict["result"])

In [ ]:
def main_agent(query: str):
    # Step 1: Check if query is relevant
    if not is_uae_real_estate_query(query):
        return {"type": "output", "data": "This is an irrelevant question to UAE property."}

    # Step 2: Extract data intent from user query
    data_intent = extract_data_intent.invoke(query)

    # Step 3: Use PandasAI to retrieve relevant data
    data_json_str = safe_dataframe_tool.invoke(data_intent)
    data_dict = json.loads(data_json_str)

    if not data_dict.get("success"):
        return {"type": "error", "error": data_dict.get("error"), "solution": data_dict.get("solution")}

    # Step 4: Classify the user's goal
    action = llm_classifier(query)
    

    # Step 5: Take action based on user intent
    if action == "output":
        return {"type": "data", "data": data_dict["result"]}
    
    
    elif action == "plot_stats":

        viz_input = json.dumps({"data": data_dict["result"], "query": query})
        result_json_str = create_plotly_code.invoke(viz_input)
        result = json.loads(result_json_str)
        return {"type": "plot", **result}

    elif action == "geospatial_plot":
        
        html = generate_google_maps_html(data_dict["result"])
        return {"type": "html", "content": html}

    else:
        return {"type": "error", "error": f"Unknown action '{action}' from classifier."}


In [ ]:
result = main_agent("show on map 2 bedroom flats that are cheaper than 2 million and in Dubai")

In [ ]:
result

In [ ]:
data_intent = extract_data_intent.invoke("create time series plot of the number of properties added in dubai per month")

    # Step 3: Use PandasAI to retrieve relevant data
data_json_str = safe_dataframe_tool.invoke(data_intent)
data_dict = json.loads(data_json_str)

In [ ]:
data_dict

In [ ]:
import streamlit as st
import pandas as pd
import openai
from typing import Optional

def get_openai_llm():
    """Initialize OpenAI client"""
    return openai.OpenAI(api_key=st.secrets["OPENAI_API_KEY"])

def extract_python_code(text: str) -> Optional[str]:
    """
    Extract Python code from markdown code blocks.
    Returns None if no code found.
    """
    if '```python' in text:
        return text.split('```python')[1].split('```')[0].strip()
    elif '```' in text:
        return text.split('```')[1].split('```')[0].strip()
    return None

def create_plotly_code(data_dict: dict, user_input: str, verbose: bool = True):
    """Generate and execute Plotly code using your exact prompt format"""
    # Prepare data
    data = pd.DataFrame(data_dict["result"])
    
    # Create prompt (using your exact format)
    code_prompt = f"""
        Generate the code <code> for plotting the data, {data}, in plotly,
        in the format requested by: {user_input}.
        The solution should be given using plotly and only plotly.
        Do not use matplotlib. Return the code <code> in the following
        format python <code>
    """
    # Get AI response
    official_ai = get_openai_llm()
    response = official_ai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": code_prompt}],
        max_tokens=1000
    )
    
    # Extract code
    raw_response = response.choices[0].message.content
    code = extract_python_code(raw_response)

    # Modify code for Streamlit
    code = code.replace("fig.show()", "")
    code += "\nst.plotly_chart(fig, use_container_width=True)"
    
    return code

In [ ]:
data_intent = extract_data_intent.invoke("make bar plot of the average property prices in Jumeirah by the number of rooms")

    # Step 3: Use PandasAI to retrieve relevant data
data_json_str = safe_dataframe_tool.invoke(data_intent)
data_dict = json.loads(data_json_str)

In [ ]:
from src.tools import get

In [ ]:
data_intent

In [ ]:
DATA_DIR = r"../data/uae_real_estate_2024_geo_ready2.parquet"
df =  pd.read_parquet(DATA_DIR)

In [ ]:
df.drop(columns="addedOn", inplace=True)

In [ ]:
df.columns

In [ ]:
df.columns[df.isna().any()].tolist()


In [ ]:
df['bathrooms'].fillna(0, inplace=True)
df['bedrooms'].fillna(0, inplace=True)
df['furnishing'] = df['furnishing'].cat.add_categories('Unknown')
df['furnishing'] = df['furnishing'].fillna('Unknown')


In [ ]:
df["sizeMin"] = df["bedrooms"].apply(lambda x: 20 + 12 * x)

In [ ]:
df['addedOn'] = df['addedOn'].dt.strftime('%Y-%m-%dT%H:%M:%S')

In [ ]:
df["displayAddress"]

In [ ]:
df.columns[df.isna().any()].tolist()


In [ ]:
df.to_csv(r"../data/uae_real_estate_2024_geo_ready2.csv", index=False)

In [ ]:
import pandasai as pai

companies = pai.create(
  path="my-org/properties3",
  df=pai.read_csv("../data/uae_real_estate_2024_geo_ready2.csv"),
  description="UAE properties dataset 2"
)

In [ ]:
result = companies.chat("Retrieve properties located within a 2-mile radius of Dubai Eye.")